In [0]:
!pip install pennylane-qchem

In [0]:
import numpy as np
import pennylane as qml
from pennylane import expval, var, device

In [0]:
name ='LiH';charge = 0; multiplicity=1; basis= 'sto-3g'; geometry = 'lih.xyz';
h, nr_qubits = qml.qchem.generate_hamiltonian(
    name,
    geometry,
    charge,
    multiplicity,
    basis,
    mapping='jordan_wigner',
    n_active_orbitals=3,
    n_active_electrons=2,
)
print(nr_qubits)

6


In [0]:
pi = np.pi
cos = np.cos
sin = np.sin
exp = np.exp

class eSWAP(qml.operation.Operation):
    num_params = 1
    num_wires = 2
    par_domain = "R"
    grad_method = "A"
    
    @staticmethod
    def _matrix(*params):
        """Returns the matrix representation of the operator for the
        provided parameter values, in the computational basis."""
        return np.array([[exp(-1j * (params[0]/2)), 0, 0, 0],
                         [0, cos(params[0]/ 2), -1j*sin(params[0]/ 2), 0],
                         [0, -1j*sin(params[0]/ 2), cos(params[0]/ 2), 0],
                         [0, 0, 0, exp(-1j*(params[0]/ 2))]])


In [0]:
dev = qml.device("default.qubit", wires=6)
dev.operations.update({"eSWAP"})

def circuit(params, wires=[0,1,2,3,4,5]):
    qml.Hadamard(wires=wires[0])
    qml.Hadamard(wires=wires[2])
    qml.Hadamard(wires=wires[4])

    qml.CNOT(wires=[wires[1], wires[0]])
    qml.CNOT(wires=[wires[3], wires[2]])
    qml.CNOT(wires=[wires[5], wires[4]])

    eSWAP(params[0], wires=[wires[1], wires[2]])
    eSWAP(params[0], wires=[wires[3], wires[4]])
    eSWAP(params[0], wires=[wires[0], wires[5]])
    eSWAP(params[0], wires=[wires[0], wires[1]])
    eSWAP(params[0], wires=[wires[2], wires[3]])
    eSWAP(params[0], wires=[wires[4], wires[5]])

    qml.SWAP(wires=[wires[5], wires[4]])
    qml.SWAP(wires=[wires[4], wires[3]])
    qml.SWAP(wires=[wires[3], wires[2]])
    qml.SWAP(wires=[wires[2], wires[1]])
    qml.SWAP(wires=[wires[1], wires[0]])

In [0]:
initial_params = np.random.uniform(0, 2*np.pi, 1)

qnodes = qml.map(circuit, h.ops, dev, measure='expval')
expval = qml.dot([h.coeffs], qnodes)

[1.01949236]


In [0]:
expval(initial_params)

array([-7.23048737])

QuantumFunctionError: ignored